**Drive link to the dataset used:**

(Please upload the files: "temp_global.csv and temp_annomaly_Aggregated_by_Year.csv" files before you start running the below cells from the drive link shared below)

https://drive.google.com/drive/folders/1ryR0snog9i-59s57e86FCJxQbchZekC1?usp=sharing

**NOTE:** Since the data has been extracted from our project dataset, the data cleaning operations were not included explicity. All those operations were done beforehand while working on the project.


**Please run all the below cells one at a time till the last cell, folder - Cluster_images and temperature_anomalies_animation.gif files will be get generated when you run cell-2 and cell-3 resp.**

# **Below cell of code to generate images of all locations using  basemap after k-means clustering**

In [1]:
!pip install basemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 860.7/860.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 35.8 MB/s eta 0:00:00


In [ ]:
import os

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import Normalize

# Assuming you have already loaded df_locations and df_global as before
country_df = pd.read_csv("/content/temp_annomaly_Aggregated_by_Year.csv")
df_locations = country_df

df_global = pd.read_csv("/content/temp_global.csv")


# Assuming you have already loaded df_locations and df_global as before

# Create a directory to save figures if it doesn't exist
output_dir = "/content/Cluster_images"
os.makedirs(output_dir, exist_ok=True)

# Loop through each year from 1950 to 2022
for year in range(1950, 2023):
    df_year = df_locations[df_locations['Year'] == year]
    global_anomaly = df_global[df_global['Year'] == year]['Anomaly'].iloc[0]

    # Normalize the anomalies for clustering
    scaler = StandardScaler()
    df_year['Anomaly_Scaled'] = scaler.fit_transform(df_year[['Anomaly']])

    # Perform K-Means Clustering (here we are using 3 for simplicity)
    kmeans = KMeans(n_clusters=3, random_state=42)
    df_year['Cluster'] = kmeans.fit_predict(df_year[['Anomaly_Scaled']])

    # Calculate the mean temperature anomaly for each cluster
    cluster_means = df_year.groupby('Cluster')['Anomaly'].mean()

    # Prepare the map plot
    plt.figure(figsize=(12, 6))
    m = Basemap(projection='mill', llcrnrlat=-90, urcrnrlat=90, llcrnrlon=-180, urcrnrlon=180, resolution='c')
    m.drawcoastlines()
    m.drawcountries()

    # Create a colormap and a normalization instance
    cmap = plt.cm.get_cmap('coolwarm')
    norm = Normalize(vmin=df_year['Anomaly'].min(), vmax=df_year['Anomaly'].max())

    # Plot each cluster with its mean anomaly as color luminance
    for cluster in range(3):  # replace 3 with optimal_clusters if needed
        cluster_data = df_year[df_year['Cluster'] == cluster]
        x, y = m(cluster_data['Longitude'].values, cluster_data['Latitude'].values)
        color = cmap(norm(cluster_means[cluster]))
        m.scatter(x, y, color=color, alpha=0.7)

    plt.title(f'K-Means Clustering of Temperature Anomalies for the Year {year}')
    plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), label='Temperature Anomaly (in degrees Fahrenheit)')

    plt.savefig(f'{output_dir}/Temperature_Anomaly_{year}.png', bbox_inches='tight')
    plt.close()  # Close the figure to free memory

# **Below cell of code to generate temp_anomalies_animation.gif**




In [11]:
from PIL import Image
import os
import imageio
import numpy as np

# Set the folder path
folder_path = '/content/Cluster_images'

# Get list of all image paths
image_files = sorted([
    os.path.join(folder_path, file_name)
    for file_name in os.listdir(folder_path)
    if file_name.startswith('Temperature_Anomaly') and file_name.endswith('.png')
])

# Load the first image to get the size
with Image.open(image_files[0]) as img:
    width, height = img.size

# Resize images and store them in a list
images = []
for file_path in image_files:
    with Image.open(file_path) as img:
        # Resize image if it's not the same size as the first one
        if img.size != (width, height):
            img = img.resize((width, height), Image.ANTIALIAS)
        images.append(img.copy())

# Save the images as an animated gif
# Calculate the duration for each frame
duration_per_frame = 300  # 500 milliseconds for 2 frames per second
imageio.mimsave('/content/temperature_anomalies_animation.gif', [np.array(img) for img in images], duration=duration_per_frame)


<ipython-input-11-71db23e5eff9>:26: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((width, height), Image.ANTIALIAS)


# **Below cell of code to generate .gif with slider (un-updated)**
---


In [7]:
# import os
# from ipywidgets import widgets, Layout, VBox
# from IPython.display import display, clear_output
# from PIL import Image
# import io

# # Set the path to the images
# image_folder = '/content/Cluster_images'

# # Get list of all image files sorted by name
# image_files = sorted([os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')])

# # Standard size to resize images
# standard_size = (600, 400)

# # Create the slider widget
# slider = widgets.IntSlider(
#     value=0,
#     min=0,
#     max=len(image_files) - 1,
#     step=1,
#     description='Select Year:',
#     layout=Layout(width='400px')  # Adjust the width to match the image width
# )

# # Placeholder for the image widget
# image_placeholder = widgets.Output()

# # Function to display an image when the slider is moved
# def on_value_change(change):
#     with image_placeholder:
#         clear_output(wait=True)
#         with open(image_files[change['new']], 'rb') as f:
#             image = Image.open(f)
#             image = image.resize(standard_size, Image.ANTIALIAS)  # Resize the image
#             display(image)

# slider.observe(on_value_change, names='value')
# display(VBox([image_placeholder, slider]))  # Slider is now below the image

# # Initialize with the first image
# on_value_change({'new': 0})


# **Below cell of code to generate .gif with slider (updated)**

In [14]:
import os
from ipywidgets import widgets, Layout, VBox
from IPython.display import display, clear_output
from PIL import Image
import io

# Assuming your image files are named '1950.png', '1951.png', etc.
image_folder = '/content/Cluster_images'
start_year = 1950

# Get list of all image files sorted by name
image_files = sorted([os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')])

# Standard size to resize images
standard_size = (600, 400)


# Placeholder for the image widget
image_placeholder = widgets.Output()

# Create the slider widget
slider = widgets.IntSlider(
    value=start_year,
    min=start_year,
    max=start_year + len(image_files) - 1,  # Assuming a sequence of years without gaps
    step=1,
    description='Select Year:',
    layout=Layout(width='400px')
)


# Function to display an image when the slider is moved
def on_value_change(change):
    year = change['new']  # This is the selected year
    image_index = year - start_year  # Calculate the index of the image
    with image_placeholder:
        clear_output(wait=True)
        with open(image_files[image_index], 'rb') as f:
            image = Image.open(f)
            image = image.resize(standard_size, Image.ANTIALIAS)
            display(image)

# Listen to changes in the slider value
slider.observe(on_value_change, names='value')

# Initial call to display the first image
on_value_change({'new': slider.value})

# Display the slider and placeholder
display(VBox([image_placeholder, slider]))
